In [1]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.p4p import P4P
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import json
import requests
import redis
import pendulum
import arrow
import imp
import time
import re

import types
import traceback 
import threading

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-software-rasterizer')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-logging')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('--ignore-certificate-errors')
browser = webdriver.Chrome(chrome_options=chrome_options)

In [3]:
broker_url = 'redis://localhost:6379/0'
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
p4p = P4P(market, market['lid'], market['lpwd'],broker_url=broker_url, headless=False, browser=browser)

. storage markets.json
.\storage\markets.json was deserialized!
E:\glitter\Eyelashes_config [] p4p_keywords_list_recording.json
E:\glitter\Eyelashes_config\p4p_keywords_list_recording.json was deserialized!
E:\glitter\Eyelashes_config [] p4p_keywords_list_monitor.json
E:\glitter\Eyelashes_config\p4p_keywords_list_monitor.json was deserialized!


In [ ]:
p4p.crawl = types.MethodType(crawl, p4p)

In [ ]:
p4p.save_crawling_result = types.MethodType(save_crawling_result, p4p)

In [ ]:
p4p.find_prices_and_sponsors = types.MethodType(find_prices_and_sponsors, p4p)

In [10]:
def find_sponsors(self, kws):

    top_sponsor = None
    sponsor_list = []

    url = 'https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&viewtype=L&CatId=&SearchText=' + re.sub(
        ' +', '+', kws)
    self.headers['Referer'] = url

    response = None
    result = None
    count_retry = 0
    max_retries = 2
    while response is None:
        try:
            count_retry = count_retry + 1
            response = requests.get(url, headers=self.headers)

            result = re.search(r'_search_result_data =(.*)page.setPageData\(_search_result_data\)', response.text,
                           re.M | re.DOTALL)

            result = re.sub(',[\n\t ]*}', '}',result.group(1))
            result = re.sub('"" === "true"', '"\\"\\" === \\"true\\""', result)

            break
        except Exception as e:
            traceback.print_exc()
            time.sleep(3)
            print(url)
            print(response.text)
            print('====================== retry in 3 seconds =================================')
            response = None
            if count_retry >= max_retries:
                return None
            else:
                continue

    obj = json.loads(result)
    items = obj['normalList']
    for idx, item in enumerate(items):
        company = {}
        if (item['isBrandAd']):
            top_sponsor = company
        elif (item['isP4p']):
            sponsor_list.append(company)
        else:
            break

        company['years'] = item['supplierYear']
        company['name'] = item['supplierName']
        company['url'] = item['supplierHref']
        if 'record' in item:
            company['record'] = []
            if 'transaction' in item['record']:
                company['record'].append(item['record']['transaction']['num'])
                company['record'].append('Transactions(6 months)')
                company['record'].append(item['record']['transaction']['conducted'])
            if 'responseRate' in item['record']:
                company['record'].append('Response Rate')
                company['record'].append(item['record']['responseRate'])

    return {'top_sponsor': top_sponsor, 'sponsor_list': sponsor_list}
p4p.find_sponsors = types.MethodType(find_sponsors, p4p)

In [ ]:
p4p.browser.set_window_size(1920, 1200)

In [ ]:
p4p.browser.save_screenshot('screenie.png')

In [4]:
group = '关注词_2'
group = '直通车高消费词'
group = '直通车App'

In [5]:
p4p.load_url()
p4p.switch_to_group(group)

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


25

In [ ]:
p4p.crawl(group=group)

In [6]:
p4p.set_sub_budget(85)

In [13]:
p4p.unset_sub_budget()

In [11]:
p4p.monitor(group=group)

Task Monitor start to run with group="直通车App"
switch to group: 直通车App
index: 0 25 > 3d mink lashes private label >  >>>>>> successful end 
index: 1 25 > bulk eyelash extension >  >>>>>> successful end 
index: 2 25 > cruelty free 3d mink eyelashes >  >>>>>> successful end 
index: 3 25 > cruelty free mink eyelashes >  >>>>>> successful end 
index: 4 25 > eyelash extension wholesale >  >>>>>> successful end 
index: 5 25 > eyelash extensions supplies >  >>>>>> successful end 
index: 6 25 > eyelash wholesale >  >>>>>> successful end 
index: 7 25 > eyelashes extension professional >  >>>>>> successful end 
index: 8 25 > individual eyelash extensions >  >>>>>> successful end 
index: 9 25 > individual eyelash extensions private label >  >>>>>> successful end 
index: 10 25 > korean eyelash extensions > 不需要改变价格 >  >>>>>> successful end 
index: 11 25 > lash extension supplies >  >>>>>> successful end 
index: 12 25 > mink eyelash wholesale > 不需要改变价格 >  >>>>>> successful end 
index: 13 25 > mink ey

In [10]:
p4p.monitor(group=group, sub_budget_limited=True)

In [ ]:
p4p.turn_all_off(group=group)

In [30]:
text = 

In [15]:
re.sub(': *"[]",', '', text)

'"needCompareOfferAttr": "" === "true", '

In [32]:
text

'"" === "true"'